In [2]:
import os
import json
import numpy as np
import pandas as pd
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [3]:
with open('./geo/wojewodztwa-min.geojson', 'r', encoding="utf8") as json_file:
    geojson = json.load(json_file)
v_id = pd.DataFrame([v['properties'] for v in geojson['features']])
v_id['nazwa'] = v_id['nazwa'].str.upper()

In [4]:
df = pd.read_csv('./data/ceidg_data_classif_cleaned.csv')

In [5]:
df.sample(4)

,Unnamed: 0,RandomDate,MonthOfStartingOfTheBusiness,QuarterOfStartingOfTheBusiness,MainAddressVoivodeship,MainAddressCounty,MainAddressTERC,CorrespondenceAddressVoivodeship,CorrespondenceAddressCounty,CorrespondenceAddressTERC,...,ShareholderInOtherCompanies,PKDMainSection,PKDMainDivision,PKDMainGroup,PKDMainClass,NoOfUniquePKDSections,NoOfUniquePKDDivsions,NoOfUniquePKDGroups,NoOfUniquePKDClasses,Target
2237064,2237064,2018-03-12,June,2,MAZOWIECKIE,WARSZAWA,1465198.0,MAZOWIECKIE,WARSZAWA,1465198.0,...,False,M,70.0,702.0,7022.0,3,6,10,12,False
1111081,1111081,2018-01-20,March,1,LUBELSKIE,LUBLIN,663011.0,LUBELSKIE,LUBLIN,663011.0,...,False,Q,86.0,862.0,8621.0,6,6,6,7,False
582344,582344,2018-06-25,July,3,LUBUSKIE,STRZELECKO-DREZDENECKI,806014.0,LUBUSKIE,STRZELECKO-DREZDENECKI,806014.0,...,False,H,49.0,494.0,4941.0,2,2,3,3,False
943486,943486,2018-08-19,April,2,ZACHODNIOPOMORSKIE,SZCZECINECKI,3215011.0,ZACHODNIOPOMORSKIE,SZCZECINECKI,3215011.0,...,False,I,56.0,561.0,5610.0,2,2,2,2,False


In [6]:
data = df[['MainAddressVoivodeship', 'PKDMainSection']]

In [7]:
data[data['PKDMainSection'] == 'M'].groupby('MainAddressVoivodeship').size().to_frame('size').reset_index()

,MainAddressVoivodeship,size
0,DOLNOŚLĄSKIE,23281
1,KUJAWSKO-POMORSKIE,10904
2,LUBELSKIE,10328
3,LUBUSKIE,5303
4,MAZOWIECKIE,70494
5,MAŁOPOLSKIE,24967
6,OPOLSKIE,4875
7,PODKARPACKIE,10179
8,PODLASKIE,6224
9,POMORSKIE,19736


In [17]:
data = df[['MainAddressVoivodeship', 'PKDMainSection']]
sections = sorted([x for x in data['PKDMainSection'].unique() if str(x) != 'nan'])
col_options = [dict(label=x, value=x) for x in sections]
dimensions = ["Sekcja"]

app = dash.Dash(
    __name__, external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"]
)

app.layout = html.Div(
    [
        html.H1("Nie umiem w html"),
        html.Div(
            [
                html.P([d + ":", dcc.Dropdown(id=d, options=col_options)])
                for d in dimensions
            ],
            style={"width": "20%", "float": "left"},
        ),
        dcc.Graph(id="graph", style={"width": "80%", "display": "inline-block"}),
    ]
)


@app.callback(Output("graph", "figure"), [Input(d, "value") for d in dimensions])
def make_figure(section):
    if section is None:
        v_size = data.groupby('MainAddressVoivodeship').size().to_frame('size').reset_index()
    else:
        v_size = data[data['PKDMainSection'] == section].groupby('MainAddressVoivodeship').size().to_frame('size').reset_index()
    map_data = pd.merge(v_size, v_id.set_index('nazwa'), left_on='MainAddressVoivodeship', right_index=True)
    fig = px.choropleth(map_data, geojson=geojson, color="size", locations="id", featureidkey="properties.id", projection="mercator", color_continuous_scale="peach", 
                        labels={'id': 'id województwa', 'size': 'liczba zarejestrowanych działalności'}, hover_name="MainAddressVoivodeship", hover_data=['size'],
                        height=800)
    fig.update_geos(fitbounds="locations", visible=False, lataxis_range=[50,60], lonaxis_range=[0, 30])
    return fig

app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/May/2020 16:35:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:35:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:35:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:35:24] "GET /_favicon.ico?v=1.11.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:35:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2020 16:35:36] "POST /_dash-update-component HTTP/1.1" 200 -
